사용 라이브러리 소개 및 데이터 전처리 과정

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
customers = pd.read_csv('./dacon_data/open/customers.csv')
locations = pd.read_csv('./dacon_data/open/locations.csv')
orders = pd.read_csv('./dacon_data/open/orders.csv')
order_items = pd.read_csv('./dacon_data/open/order_items.csv')
payments = pd.read_csv('./dacon_data/open/payments.csv')
products = pd.read_csv('./dacon_data/open/products.csv')
reviews = pd.read_csv('./dacon_data/open/reviews.csv')
sellers = pd.read_csv('./dacon_data/open/sellers.csv')
# temp = pd.merge(reviews,order_items, how='inner',on='Order_id')
# temp = pd.merge(temp,orders, how='inner',on='Order_id')
# temp = pd.merge(temp,payments, how='inner',on='Order_id')
# temp = pd.merge(temp,products, how='inner',on='Product_id')
# temp = pd.merge(temp,customers, how='inner',on='Customer_id')
# temp = pd.merge(temp,sellers, how='inner',on='Seller_id')
# temp = temp.drop(["Review_creation_date","Review_answer_timestamp","Order_status","Payment_sequential","Payment_type","Payment_installments"],axis=1)
# temp = temp.drop(["Product_weight_g","Product_length_cm","Product_height_cm","Customer_zipcode_prefix","Seller_zipcode_prefix"],axis=1)
# temp = temp.drop(["Product_width_cm"],axis=1)

In [63]:
reviews.head(1)

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59


In [64]:
reviews_groupby = reviews.groupby(["Order_id"])["Review_score"].mean().reset_index()

In [65]:
temp = pd.merge(reviews_groupby,order_items, how='inner',on='Order_id')

In [66]:
temp

,Order_id,Review_score,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_00000,4.0,1,PRODUCT_02093,SELLER_0491,29.99,8.72
1,ORDER_00001,5.0,1,PRODUCT_26747,SELLER_2662,45.00,27.20
2,ORDER_00002,5.0,1,PRODUCT_10598,SELLER_1354,19.90,8.72
3,ORDER_00003,4.0,1,PRODUCT_16703,SELLER_1518,147.90,27.36
4,ORDER_00004,5.0,1,PRODUCT_08274,SELLER_1599,59.99,15.17
...,...,...,...,...,...,...,...
99772,ORDER_88084,4.0,1,PRODUCT_01588,SELLER_0031,174.90,20.10
99773,ORDER_88085,5.0,1,PRODUCT_13335,SELLER_1892,205.99,65.02
99774,ORDER_88086,2.0,1,PRODUCT_07372,SELLER_2445,179.99,40.59
99775,ORDER_88086,2.0,2,PRODUCT_07372,SELLER_2445,179.99,40.59


In [67]:
# temp.loc[temp['Order_id']=='ORDER_14054']

In [68]:
temp = pd.merge(temp,orders, how='inner',on='Order_id')
temp = pd.merge(temp,payments, how='inner',on='Order_id')
temp = pd.merge(temp,products, how='inner',on='Product_id')
temp = pd.merge(temp,customers, how='inner',on='Customer_id')
temp = pd.merge(temp,sellers, how='inner',on='Seller_id')

우편번호 앞자리가 동일한 경우 평균 위도 경도를 구하여 해당 주소로 임의의 위치를 선정하고자 함

In [69]:
locations_group_lat=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lat"].mean().reset_index()
locations_group_lng=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lng"].mean().reset_index()
locations_group =  pd.merge(locations_group_lat, locations_group_lng, how = 'inner', on="Geolocation_zipcode_prefix")

소비자 위도 경도 컬럼명 변경

In [70]:
temp = pd.merge(temp, locations_group, left_on = 'Customer_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Customer_lat',"Geolocation_lng" : "Customer_lng" })

판매자 위도 경도 merge 후 컬럼명 변경

In [71]:
temp = pd.merge(temp, locations_group, left_on = 'Seller_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Seller_lat',"Geolocation_lng" : "Seller_lng" })

In [72]:
temp["Distance"] = ((temp["Customer_lat"] - temp["Seller_lat"])**2 + (temp["Customer_lng"] - temp["Seller_lng"])**2) **(1/2)

to_datetime 형태로 바꾸겠단 소리임

In [73]:
temp['Order_purchase_timestamp']=pd.to_datetime(temp['Order_purchase_timestamp'])
temp['Order_delivered_carrier_date']=pd.to_datetime(temp['Order_delivered_carrier_date'])
temp['Order_delivered_customer_date']=pd.to_datetime(temp['Order_delivered_customer_date'])
temp['Order_estimated_delivery_date']=pd.to_datetime(temp['Order_estimated_delivery_date'])

In [74]:
# 구매할때부터 물류처리할때까지 걸리는 시간
temp["time_diff_purchase_carrier"] = temp["Order_delivered_carrier_date"] - temp["Order_purchase_timestamp"]
# 구매할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_purchase_customer"] = temp["Order_delivered_customer_date"] - temp["Order_purchase_timestamp"]
# 물류처리할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_carrier_customer"] = temp["Order_delivered_customer_date"] - temp["Order_delivered_carrier_date"]
# 기대배송날짜와 실제배송날짜의 차이
temp["time_diff_customer_delivery"]= temp['Order_estimated_delivery_date'] - temp["Order_delivered_customer_date"]

시간을 초로 변환

In [75]:
temp["time_diff_purchase_carrier_second"] = temp["time_diff_purchase_carrier"].apply(lambda x : x.total_seconds())
temp["time_diff_purchase_customer_second"] = temp["time_diff_purchase_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_carrier_customer_second"] = temp["time_diff_carrier_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_customer_delivery_second"] = temp["time_diff_customer_delivery"].apply(lambda x : x.total_seconds())

In [76]:
temp['Product_weight_g'] = temp['Product_weight_g'].replace('Unknown', np.nan).fillna(0).astype(float)
temp['YearMonth'] = temp['Order_purchase_timestamp'].dt.strftime('%Y%m') #월별로 분류

이거 그냥 피클에 넣겠단 소리임

In [77]:
import pickle
with open("./dacon_total_df.pkl", "wb") as f:
    pickle.dump(temp,f)

In [78]:
# group by 용 Product_category_name                   home_utilities
# 정규화

In [79]:
# temp.columns

In [80]:
# product_id를 가져오자

In [ ]:
# 'Payment_value': 1,
#     'Review_score': 0.8,
#     'Price': 0.6,
#     'time_diff_customer_delivery_second': 0.5,
#     'Distance': 0.4,
#     'Freight_value': 0.3,
#     'time_diff_purchase_customer_second': 0.2,

In [149]:
temp.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Review_score,Order_item_id,Price,Freight_value,Payment_sequential,Payment_installments,Payment_value,Product_weight_g,Customer_zipcode_prefix,Seller_zipcode_prefix,Geolocation_zipcode_prefix_x,Customer_lat,Customer_lng,Geolocation_zipcode_prefix_y,Seller_lat,Seller_lng,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
Review_score,1.000000,-0.136631,0.004038,-0.036344,0.009013,-0.041931,-0.082200,-0.024695,-0.030842,0.030551,-0.030842,-0.037966,-0.042776,0.030551,-0.007809,0.012715,-0.059338,-0.149873,-0.309528,-0.276505,0.243981
Order_item_id,-0.136631,1.000000,-0.060882,-0.026244,-0.000802,0.072410,0.256940,-0.002340,-0.004303,-0.018655,-0.004303,-0.018944,-0.018802,-0.018655,-0.005989,-0.010462,-0.020506,0.048714,-0.016850,-0.038663,0.019393
Price,0.004038,-0.060882,1.000000,0.410490,0.000156,0.279295,0.732875,0.340641,0.041878,0.087132,0.041878,0.055736,0.015998,0.087132,0.024830,0.046537,0.084910,0.063440,0.061040,0.040461,0.001780
Freight_value,-0.036344,-0.026244,0.410490,1.000000,0.009163,0.186932,0.368741,0.615169,0.225188,0.147593,0.225188,0.274120,0.092461,0.147593,0.036002,-0.004605,0.389559,0.093456,0.220277,0.202256,0.037662
Payment_sequential,0.009013,-0.000802,0.000156,0.009163,1.000000,-0.089320,-0.065962,0.028461,-0.031319,0.005551,-0.031319,-0.011271,0.004977,0.005551,0.006480,-0.001422,-0.014118,0.011558,0.000573,-0.004171,-0.000903
Payment_installments,-0.041931,0.072410,0.279295,0.186932,-0.089320,1.000000,0.268191,0.181103,0.056723,0.036800,0.056723,0.059638,0.030472,0.036800,0.009270,-0.010432,0.083342,-0.001386,0.043322,0.047984,0.032023
Payment_value,-0.082200,0.256940,0.732875,0.368741,-0.065962,0.268191,1.000000,0.302633,0.049997,0.067348,0.049997,0.058305,0.015592,0.067348,0.012298,0.025748,0.086942,0.095294,0.059627,0.025690,0.014989
Product_weight_g,-0.024695,-0.002340,0.340641,0.615169,0.028461,0.181103,0.302633,1.000000,0.001849,0.011210,0.001849,0.002381,0.001332,0.011210,-0.022212,0.011481,-0.008067,0.148066,0.081033,0.027208,-0.011399
Customer_zipcode_prefix,-0.030842,-0.004303,0.041878,0.225188,-0.031319,0.056723,0.049997,0.001849,1.000000,0.071668,1.000000,0.115885,-0.312244,0.071668,-0.011225,-0.016091,0.489961,0.010590,0.275780,0.297396,0.060686
Seller_zipcode_prefix,0.030551,-0.018655,0.087132,0.147593,0.005551,0.036800,0.067348,0.011210,0.071668,1.000000,0.071668,-0.036958,-0.035919,1.000000,-0.151135,-0.431021,0.172356,0.002455,0.044854,0.048065,0.123175


구매자가 물건을 구매하는 데 있어서 고려하는 여러 가지 평가 지표를 하나의 df 뽑았다.
(이 아래는 아마 분석 목적으로 들어 갈 것 같음)  
가중치를 잡아서 필요한 컬럼을 뽑아서 가중치를 부여한 다음에 고객에게 최고의 물건을 판매한다는 가정 하에  
회사에서도 매출을 최대한 올릴 수 있는 방법을 위해서

In [81]:
df = temp[["Product_id","Review_score","Price","Freight_value","time_diff_purchase_customer_second","time_diff_customer_delivery_second","Distance","Payment_value"]]

위에 셀러 아이디 추가해야 될 거 같은데?  
아래에 있는 거 보니까 구분이 안 되는 거 같아서

Product_id 가 겹치는 것이 있는 가 확인을 하고

In [82]:
product_id_counts = df['Product_id'].value_counts()

Product_id 1개인 경우 판매자가 하나인 것이기 때문에 고려하지 않도록 아래 코드 실행  
판매자가 하나 뿐이라면 이미 독점을 하고 있어서 매출을 올릴 수 있는 방법에 대해 고려할 필요가 없음

In [83]:
product_id_up_2 = product_id_counts[product_id_counts >= 2].index

위의 과정 df 로 정의한 거

In [84]:
choose_product_id = df[df['Product_id'].isin(product_id_up_2)]

아래는 예시 보기

In [85]:
choose_product_id[choose_product_id.Product_id=="PRODUCT_01923"]

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
100532,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,82.38
100533,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,28.89
100534,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,8.98


In [86]:
product_id_no_dup = choose_product_id.drop_duplicates()

In [87]:
# unique_rows.Product_id.value_counts()

In [88]:
product_id_counts1 = product_id_no_dup['Product_id'].value_counts()

In [89]:
product_id_up_2_1 = product_id_counts1[product_id_counts1 >= 2].index

In [90]:
choose_product_id_1 = product_id_no_dup[product_id_no_dup['Product_id'].isin(product_id_up_2_1)]

In [91]:
dacon_last_df = choose_product_id_1.sort_values(by='Product_id')

In [92]:
# temp.columns

가중치 분배하기  
- 추후 수정 예정

In [93]:
dacon_last_df.columns


Index(['Product_id', 'Review_score', 'Price', 'Freight_value',
       'time_diff_purchase_customer_second',
       'time_diff_customer_delivery_second', 'Distance', 'Payment_value'],
      dtype='object')

time_diff_customer_delivery_second , time_diff_purchase_customer_second 는 한 번 구매해 본 사람만이 고려할 사항이라 낮게 비중을 둬도 될 거 같음

In [94]:
variable_importance = {
    'Payment_value': 1,
    'Review_score': 0.8,
    'Price': 0.6,
    'time_diff_customer_delivery_second': 0.5,
    'Distance': 0.4,
    'Freight_value': 0.3,
    'time_diff_purchase_customer_second': 0.2,
}

In [95]:
total_importance = sum(variable_importance.values())

가중치 중요도 계산(퍼센트로)

In [96]:
normalized_importance = {key: value / total_importance for key, value in variable_importance.items()}

In [97]:
normalized_importance

{'Payment_value': 0.2631578947368421,
 'Review_score': 0.2105263157894737,
 'Price': 0.15789473684210525,
 'time_diff_customer_delivery_second': 0.13157894736842105,
 'Distance': 0.10526315789473685,
 'Freight_value': 0.07894736842105263,
 'time_diff_purchase_customer_second': 0.052631578947368425}

In [98]:
dacon_last_df.Product_id.unique()

array(['PRODUCT_00005', 'PRODUCT_00006', 'PRODUCT_00009', ...,
       'PRODUCT_29466', 'PRODUCT_29468', 'PRODUCT_29469'], dtype=object)

예시 보려고 test_df 정의

In [99]:
test_df = dacon_last_df[dacon_last_df.Product_id=="PRODUCT_29468"]

In [100]:
test_df

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
47605,PRODUCT_29468,5.0,127.5,17.14,786643.0,1144294.0,3.012221,144.64
597,PRODUCT_29468,5.0,127.5,11.91,142457.0,1086465.0,0.649095,139.41
52932,PRODUCT_29468,4.0,127.5,11.91,434148.0,1070669.0,0.204524,139.41
44554,PRODUCT_29468,5.0,127.0,12.39,1129132.0,608472.0,6.256644,139.39
39302,PRODUCT_29468,4.0,127.0,34.69,2753075.0,-76352.0,16.906066,161.69


문자열이라서 정규화 시킬 수 없어서 일시적으로 제외한 것입니다~

In [101]:
test_df = test_df.drop(["Product_id"],axis=1)

test_df 정규화 해보는 것임!

In [102]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(test_df)
X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
# print(X_standardized_df.Price)

In [103]:

for column, weight in normalized_importance.items():
    X_standardized_df[column] *= weight

In [104]:
# print(X_standardized_df)
weighted_sum = X_standardized_df.sum(axis=1)

In [105]:
weighted_sum.max()

0.33952241169598857

가중치 넣은 거 순서대로 순위 부여한 것임

In [106]:
# Payment_value' 1등 
# 'Review_score', 2등  
# 'Price', ' 3등  
# 'time_diff_customer_delivery_second',  4등  
# Distance', ' 5등 	 
# Freight_value', 6등 
# time_diff_purchase_customer_second', 7등

test_df 가중치 돌린 거 하나로 합쳐둔 것

In [107]:
#전체다시 합할 df
total_df=pd.DataFrame()
# 가중치
# relative_importance
# 비중 normalized_importance
# test_df = sorted_data[sorted_data.Product_id=="PRODUCT_29466"]
# test_df = test_df.drop(["Product_id"],axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(test_df)
X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
# print(X_standardized_df.Price)
# X_standardized_df
for column, weight in normalized_importance.items():
    X_standardized_df[column] *= weight
# print(X_standardized_df)
weighted_sum = X_standardized_df.sum(axis=1)
# weighted_sum

for 문에 넣을 list 만든 것임

In [108]:
dacon_last_id = dacon_last_df['Product_id'].unique()

In [109]:
len(dacon_last_id) #12517개

12517

In [110]:
dacon_last_df.head(2)

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
67595,PRODUCT_00005,2.0,958.0,27.76,468466.0,1560059.0,4.628797,985.76
64159,PRODUCT_00005,5.0,968.0,44.30,1151056.0,1314046.0,7.932873,1012.30


파일 손상 고려해서 복사본으로 재정의

In [111]:
# dacon_last_df 원본파일
dacon_remake_df = dacon_last_df.copy()

동일한 지 확인하려고?

In [112]:
dacon_last_df.Payment_value.sum() 
# 11288914.88

11288914.88

구하고자 하는 거 찐으로 가중치 돌리기  
dacon_remake_df 로 한 거임  
주석처리 된 거 해제하고 코드 실행해야 함

Payment_value 는 가중치 값이 가장 높은 것과 동일하게 뽑는 과정이 포함되어 있고  
이는 해당 상품을 가중치 높은 것으로 모두 판매 했을 경우 매출액을 보기 위한 목적

In [113]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
i =0
for product_id in tqdm(dacon_last_id):
    test_df = dacon_last_df[dacon_last_df['Product_id'] == product_id]
    test_df = test_df.drop(["Product_id"],axis=1)
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(test_df)
    X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
    for column, weight in normalized_importance.items():
        X_standardized_df[column] *= weight
    weighted_sum = X_standardized_df.sum(axis=1)
    weighted_sum_max=weighted_sum.max()
    max_index = weighted_sum.idxmax()
    # print(product_id +"    "+ str(max_index))
    # print(test_df.iloc[max_index]["Payment_value"])
    max_payment_value=test_df.iloc[max_index]["Payment_value"]
    # dacon_remake_df[dacon_remake_df['Product_id'] == product_id].Payment_value = max_payment_value
    dacon_remake_df.loc[dacon_remake_df["Product_id"]==product_id , "Payment_value"] = max_payment_value
    # print(dacon_remake_df.head(2))
    i=i+1
    # break

100%|████████████████████████████████████████████████████████████████████████████| 12517/12517 [02:02<00:00, 101.97it/s]


In [148]:
dacon_last_df[dacon_last_df.Product_id=="PRODUCT_00032"]

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
6775,PRODUCT_00032,5.0,103.37,13.28,739437.0,299875.0,0.132714,116.65
96090,PRODUCT_00032,3.0,103.37,26.80,866920.0,1059689.0,6.615789,260.34


제대로 실행 되었는가 확인하기 위함

In [147]:
dacon_remake_df[dacon_remake_df.Product_id=="PRODUCT_00032"]

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
6775,PRODUCT_00032,5.0,103.37,13.28,739437.0,299875.0,0.132714,260.34
96090,PRODUCT_00032,3.0,103.37,26.80,866920.0,1059689.0,6.615789,260.34


In [146]:
dacon_remake_df.Product_id.unique().tolist() 
# PRODUCT_00032 PRODUCT_00034 PRODUCT_00035

['PRODUCT_00005',
 'PRODUCT_00006',
 'PRODUCT_00009',
 'PRODUCT_00010',
 'PRODUCT_00014',
 'PRODUCT_00015',
 'PRODUCT_00016',
 'PRODUCT_00019',
 'PRODUCT_00023',
 'PRODUCT_00025',
 'PRODUCT_00026',
 'PRODUCT_00030',
 'PRODUCT_00032',
 'PRODUCT_00034',
 'PRODUCT_00035',
 'PRODUCT_00036',
 'PRODUCT_00039',
 'PRODUCT_00041',
 'PRODUCT_00042',
 'PRODUCT_00044',
 'PRODUCT_00057',
 'PRODUCT_00060',
 'PRODUCT_00061',
 'PRODUCT_00064',
 'PRODUCT_00065',
 'PRODUCT_00069',
 'PRODUCT_00070',
 'PRODUCT_00071',
 'PRODUCT_00072',
 'PRODUCT_00074',
 'PRODUCT_00075',
 'PRODUCT_00083',
 'PRODUCT_00085',
 'PRODUCT_00093',
 'PRODUCT_00097',
 'PRODUCT_00099',
 'PRODUCT_00101',
 'PRODUCT_00102',
 'PRODUCT_00103',
 'PRODUCT_00104',
 'PRODUCT_00105',
 'PRODUCT_00106',
 'PRODUCT_00107',
 'PRODUCT_00111',
 'PRODUCT_00113',
 'PRODUCT_00114',
 'PRODUCT_00123',
 'PRODUCT_00124',
 'PRODUCT_00125',
 'PRODUCT_00126',
 'PRODUCT_00132',
 'PRODUCT_00134',
 'PRODUCT_00135',
 'PRODUCT_00137',
 'PRODUCT_00138',
 'PRODUCT_

확인용..  
제대로 들어갔을까?

In [115]:
print(i)

12517


In [116]:
dacon_remake_df.Payment_value.sum() 
# 19173054.28

19173054.28

이익 봤을 금액

In [117]:
dacon_remake_df.Payment_value.sum()  - dacon_last_df.Payment_value.sum()

7884139.4

In [118]:
dacon_last_df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
Review_score,1.000000,-0.000591,-0.035050,-0.329817,0.269120,-0.063475,-0.051440
Price,-0.000591,1.000000,0.409349,0.059547,0.003765,0.082755,0.868687
Freight_value,-0.035050,0.409349,1.000000,0.229756,0.040730,0.409169,0.426265
time_diff_purchase_customer_second,-0.329817,0.059547,0.229756,1.000000,-0.647809,0.399406,0.066095
time_diff_customer_delivery_second,0.269120,0.003765,0.040730,-0.647809,1.000000,0.055681,0.018912
Distance,-0.063475,0.082755,0.409169,0.399406,0.055681,1.000000,0.101854
Payment_value,-0.051440,0.868687,0.426265,0.066095,0.018912,0.101854,1.000000


In [119]:
dacon_remake_df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
Review_score,1.000000,-0.000591,-0.035050,-0.329817,0.269120,-0.063475,-0.027749
Price,-0.000591,1.000000,0.409349,0.059547,0.003765,0.082755,0.643344
Freight_value,-0.035050,0.409349,1.000000,0.229756,0.040730,0.409169,0.304476
time_diff_purchase_customer_second,-0.329817,0.059547,0.229756,1.000000,-0.647809,0.399406,0.083154
time_diff_customer_delivery_second,0.269120,0.003765,0.040730,-0.647809,1.000000,0.055681,-0.013403
Distance,-0.063475,0.082755,0.409169,0.399406,0.055681,1.000000,0.070825
Payment_value,-0.027749,0.643344,0.304476,0.083154,-0.013403,0.070825,1.000000


kpi: 가중치를 비교했을 때 가장 높은 값이 나오는 payment_value 가 나오도록 제품을 판매하자  
그럼 위를 실행하기 위해서 제안은 어떤걸 할건데?

가까운 가격이 소비자에게 노출되도록, 상단 노출은 아니더라도 에디터 픽 이런 식으로 추천이 뜨도록